# Эксперименты с тональным словарем и rule-based методами

## Функции обработки текста

### Исправление опечаток

In [86]:
!pip install pyaspeller

In [87]:
from pyaspeller import YandexSpeller

def spellcheck(text):
    try:
        speller = YandexSpeller()
        changes = {change['word']: change['s'][0] for change in speller.spell(text)}
        for word, suggestion in changes.items():
            text = text.replace(word, suggestion)
        return text
    except:
        return 'error'

In [88]:
spellcheck('лучьше')

'лучше'

### Замена ё  → е

In [89]:
def delete_yo(text):
    text = text.lower()
    text = text.replace('ё', 'е')
    return text

In [90]:
delete_yo('ёлка')

'елка'

## Тональный словарь для русского

### Для определения тональности отдельных слов был взят тональный словарь для русского [Карта слов](https://github.com/dkulagin/kartaslov/tree/master/dataset/kartaslovsent).

In [91]:
import csv

In [92]:
from google.colab import files
uploaded = files.upload()

Saving kartaslovsent.csv to kartaslovsent (1).csv


In [93]:
with open('kartaslovsent.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    data = []
    for row in spamreader:
        data.append(row)

In [94]:
data[:5]

[['term',
  'tag',
  'value',
  'pstv',
  'ngtv',
  'neut',
  'dunno',
  'pstvNgtvDisagreementRatio'],
 ['абажур', 'NEUT', '0.08', '0.185', '0.037', '0.58', '0.198', '0.0'],
 ['аббатство', 'NEUT', '0.1', '0.192', '0.038', '0.578', '0.192', '0.0'],
 ['аббревиатура', 'NEUT', '0.08', '0.196', '0.0', '0.63', '0.174', '0.0'],
 ['абзац', 'NEUT', '0.0', '0.137', '0.0', '0.706', '0.157', '0.0']]

In [95]:
len(data)

46128

In [96]:
# замняем ё на е в словаре (в отзывах и так нет ё)
dct_sentiment = {}
for i in data:
  dct_sentiment[delete_yo(i[0])] = i[1:]

In [97]:
dct_sentiment['елка']

['PSTV', '0.59', '0.492', '0.033', '0.385', '0.09', '0.0']

## Отзывы

In [98]:
import pandas as pd
from nltk.tokenize import sent_tokenize
import numpy as np
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [99]:
from torch.utils.data import Dataset, DataLoader
import torch
import warnings

In [100]:
ans = pd.read_csv('https://raw.githubusercontent.com/PhilBurub/NLPcourse_HSE/main/train_aspects.txt', header=None, delimiter='\t', index_col=0,
                  names=['aspect', 'entity', 'start', 'end', 'sentiment'])
texts = pd.read_csv('https://raw.githubusercontent.com/PhilBurub/NLPcourse_HSE/main/train_reviews.txt', header=None, delimiter='\t', index_col=0,
                    names=['text'])

In [101]:
def boudaries(id_, text):
  out = []
  new_text = text
  cur = 0
  for sent in sent_tokenize(text):
    start = new_text.find(sent)
    end = start + len(sent)
    out.append((id_, sent, cur + start, cur + end))
    cur += end
    new_text = new_text[end:]
  return pd.DataFrame(out, columns=['id', 'text', 'start', 'end'])

In [102]:
class ContextDataset(Dataset):
  def __init__(self, texts, outputs):

    new = pd.DataFrame()
    for id_, text in texts.iterrows():
      new = pd.concat((new, boudaries(id_, text['text'])))

    out = []
    for text_id, row in outputs.iterrows():
      slice_ = new[(new['id'] == text_id) & (new['start'] <= row['start']) & (new['end'] >= row['end'])]
      if len(slice_) == 0:
        print(row, '0')
        continue
      if len(slice_) > 1:
        print(row, '>1')
        continue
      out.append((text_id, row['entity'], row['aspect'], slice_['text'].item(), row['sentiment']))
    self.contexts = pd.DataFrame(out, columns=['id', 'input', 'aspect', 'context', 'sentiment'])

    self.map_sent = {0: -1, 'negative': 0, 'neutral': 1, 'positive': 2,
                     'both': 3}
    self.map_asp = {0: -1, 'Food': 0, 'Interior': 1, 'Price': 2,
                    'Service': 3, 'Whole': 4}

  def __len__(self):
    return len(self.contexts)

  def __getitem__(self, idx):
    if isinstance(idx, slice):
      one = False
    else:
      one = True

    row = self.contexts.loc[idx]
    with warnings.catch_warnings():
      warnings.simplefilter("ignore")
      if one:
        row.loc['CLS'] = '[CLS]'
      else:
        row.loc[:, 'CLS'] = '[CLS]'
    sent = row[['input', 'CLS', 'context']].values
    sentence = ' '.join(sent) if one else list(map(lambda x: ' '.join(x), sent))
    aspect = [self.map_asp.get(row['aspect'])] if one else \
                              row['aspect'].apply(self.map_asp.get).tolist()
    sentiment = [self.map_sent.get(row['sentiment'])] if one else \
                              row['sentiment'].apply(self.map_sent.get).tolist()
    return sentence, torch.tensor(aspect, dtype=torch.int32), torch.tensor(sentiment, dtype=torch.int32)

In [103]:
train_set = ContextDataset(texts, ans)

In [104]:
train_set.contexts.head(5)

,id,input,aspect,context,sentiment
0,3976,ресторане,Whole,Решил написать отзыв о ресторане в котором отм...,neutral
1,3976,ресторанах,Whole,Решил написать отзыв о ресторане в котором отм...,neutral
2,3976,ресторане,Whole,Но теперь о ресторане.,neutral
3,3976,Столик бронировали,Service,Столик бронировали заранее и сделали так как п...,neutral
4,3976,администратор,Service,Столик бронировали заранее и сделали так как п...,positive


In [105]:
train_set.contexts.to_csv('reviews.csv', sep='|', index=False)

In [ ]:
files.download('reviews.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [106]:
with open('reviews.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='|')
    reviews = []
    for row in spamreader:
        reviews.append(row)

In [107]:
reviews[2]

['3976',
 'ресторанах',
 'Whole',
 'Решил написать отзыв о ресторане в котором отметили прекрасный весений праздник, прочитал отзывы edik077 и Rules77777и понял что либо мы были вразных ресторанах, либо у ребят что-то незаладилось.',
 'neutral']

## Лемматизация отзывов

In [108]:
from tqdm import tqdm

In [109]:
import pymystem3
mystem=pymystem3.Mystem()

In [110]:
! pip install stop-words

In [111]:
from stop_words import get_stop_words

In [112]:
stop_words = get_stop_words('ru')

In [115]:
# оставляем эти слова для некоторых экспериментов
stop_words.remove('не')
stop_words.remove('нет')
stop_words.remove('спасибо')

In [114]:
# лемматизация
def reviews_preparation(li_revievs):
  lem_reviews = []
  for review in tqdm(li_revievs):
    rev = review
    rev[1] = ''.join(mystem.lemmatize(rev[1])).strip('\n')

    text = rev[3].split()
    clean = []
    for w in text:
      w = w.rstrip()
      w = w.strip('.,:;?!\"\')(#·№*@\/-')
      # исправление опечаток занимает много времени, лучше применять на небольших данных
      #w = spellcheck(w)
      if w not in stop_words and w.isalpha():
        clean.append(w)
    clean_text = ' '.join(clean)
    rev[3] = ''.join(mystem.lemmatize(clean_text))
    rev[3] = rev[3].strip('\n')
    lem_reviews.append(rev)

  return lem_reviews

In [ ]:
part1 = reviews_preparation(reviews[1:800]) # с исправлением опечаток (шестая часть отзывов)

100%|██████████| 799/799 [2:43:18<00:00, 12.26s/it]


In [ ]:
with open('reviews_lem.csv', 'w', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(reviews[0])
    csvwriter.writerows(part1)

In [ ]:
files.download('reviews_lem.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [116]:
lem_reviews = reviews_preparation(reviews) # без исправления опечаток (все отзывы train)

100%|██████████| 4764/4764 [00:04<00:00, 952.93it/s] 


In [117]:
answers = []
for i in lem_reviews:
  answers.append(i[4])

## Эксперименты
### (От наименее удачных к наиболее удачным)

#### №1. Считаем эмоционально-оценочный заряд всего контекста (складываем заряды всех слов и делим на количество слов в контексте (таргетное слово не учитывается)).

In [118]:
def mean_sentiment(texts):
  res = []
  zero_contexts = 0
  zero_texts = []
  for i in texts:

    target_word = i[1].split()
    context = i[3].split()

    context_score = 0
    known = 0

    for word in context:
      if word not in target_word and word in dct_sentiment.keys():
        known += 1
        context_score += float(dct_sentiment[word][1])

    if known == 0:
      zero_contexts += 1
      zero_texts.append(i[3])

    if known > 0:
      res.append(context_score / known)
    else:
      res.append(0)

  res_mapping = []
  for j in res:
    if j >= 0.09:
      res_mapping.append('positive')
    elif j <= 0.03:
      res_mapping.append('negative')
    else:
      res_mapping.append('neutral')

  return res_mapping

In [119]:
from sklearn.metrics import accuracy_score

In [120]:
res_mean_sentiment = mean_sentiment(lem_reviews)

In [121]:
accuracy_score(answers, res_mean_sentiment)

0.6488245172124265

Результат со спеллчекером приведен ниже. Можно заметить, что точность предсказаний с исправлением опечаток чуть ниже, чем точность без исправления. Это может быть связано с тем, что какие-то слова исправляются неверно. В следующих экспериментах мы используем полный набор данных без применения функции исправления ошибок.

In [125]:
upl = files.upload()

Saving reviews_lem.csv to reviews_lem (1).csv


In [126]:
with open('reviews_lem.csv', 'r', encoding='utf-8') as f:
  reader = csv.reader(f)
  spell_checked = []
  for row in reader:
        spell_checked.append(row)

In [127]:
answers_part1 = []
for i in spell_checked:
  answers_part1.append(i[4])

In [128]:
res_spelled = mean_sentiment(spell_checked)

In [129]:
accuracy_score(answers_part1, res_spelled) # точность со спеллчекером

0.60875

In [130]:
accuracy_score(answers[:800], res_mean_sentiment[:800]) # точность без спеллчекера

0.6125

#### №2. Считаем теги из тонального словаря за -1 (отрицательная оценка), 0 (нейтральная оценка) и 1 (положительная оценка), на основе итоговой суммы приписываем ответ.

In [131]:
def dict_tags(texts):
  res = []
  for i in texts:

    target_word = i[1].split()
    context = i[3].split()

    context_score = 0

    for word in context:
      if word not in target_word and word in dct_sentiment.keys():
        if dct_sentiment[word][0] == 'PSTV':
          context_score += 1
        elif dct_sentiment[word][0] == 'NGTV':
          context_score -= 1

    if context_score > 0:
      res.append('positive')
    elif context_score < 0:
      res.append('negative')
    else:
      res.append('neutral')
  return res

In [132]:
res_dict_tags = dict_tags(lem_reviews)

In [133]:
accuracy_score(answers, res_dict_tags) # точность

0.624895046179681

#### №3. Берем два "самых негативных" и два "самых позитивных" слова из всего контекста (то есть те слова из контекста, у которых наибольшее/наименьшее значение по тональному словарю). Если сумма значений положительно окрашенных слов больше по модулю суммы значений отрицательно окрашенных слов, то считаем, что контекст содержит позитивное отношение, если сумма меньше -- контекст содержит негативное отношение. Если разница незначительна, контекст нейтрален.

#### По отзывам заметно, что, когда посетитель положительно относится к заведению, в конце текста часто есть слова благодарности. Как правило, это гарантирует тег "positive" (поэтому контексты со словами "спасибо" и "благодарность" учитываются отдельно).

In [134]:
def max_min_sentiment(texts):
  res = []

  for i in texts:

    target_word = i[1].split()
    context = i[3].split()

    context_score = 0

    all_scores = []

    thanks = False
    for word in context:
      if word in ['спасибо', 'благодарность']:
        thanks = True
      if word not in target_word and word in dct_sentiment.keys():
        context_score += float(dct_sentiment[word][1])
        all_scores.append(float(dct_sentiment[word][1]))

    if thanks == True:
      res.append('positive')
    else:
      all_scores.sort()
      if len(all_scores) > 1:
        max_score = all_scores[-1]
        mx_score = all_scores[-2]
        min_score = all_scores[0]
        mn_score = all_scores[1]
      elif len(all_scores) == 1:
        max_score = all_scores[-1]
        mx_score = 0
        min_score = all_scores[0]
        mn_score = 0
      else:
        max_score = 0
        mx_score = 0
        min_score = 0
        mn_score = 0
      if max_score + mx_score >= -(min_score + mn_score):
        res.append('positive')
      elif max_score + mx_score >= -(min_score + mn_score) + 0.02:
        res.append('neutral')
      else:
        res.append('negative')

  return res

In [135]:
res_max_min = max_min_sentiment(lem_reviews)

In [136]:
accuracy_score(answers, res_max_min) # точность

0.6582703610411419

#### №4. Проделываем то же, что в предыдущем способе, но если непосредственно перед словом или после слова есть "не", "нет", то меняем значений из тонального словаря на противоположное по знаку.

In [137]:
# лучшая точность
def negation_invert(texts):
  res = []
  for i in texts:

    target_word = i[1].split()
    context = [0] + i[3].split() + [0]

    all_scores = []

    thanks = False
    c = 0
    for word in context[1:len(context)-1]:
      c += 1
      if word in ['спасибо', 'благодарность']:
        thanks = True
      if word not in target_word and word in dct_sentiment.keys():
        if context[c-1] in ['не', 'нет'] or context[c+1] in ['не', 'нет']:
          all_scores.append(-float(dct_sentiment[word][1]))
        else:
          all_scores.append(float(dct_sentiment[word][1]))

    if thanks == True:
      res.append('positive')
    else:
      all_scores.sort()
      if len(all_scores) > 1:
        max_score = all_scores[-1]
        mx_score = all_scores[-2]
        min_score = all_scores[0]
        mn_score = all_scores[1]
      elif len(all_scores) == 1:
        max_score = all_scores[-1]
        mx_score = 0
        min_score = all_scores[0]
        mn_score = 0
      else:
        max_score = 0
        mx_score = 0
        min_score = 0
        mn_score = 0
      if max_score + mx_score >= -(min_score + mn_score):
        res.append('positive')
      elif max_score + mx_score >= -(min_score + mn_score) + 0.03:
        res.append('neutral')
      else:
        res.append('negative')

  return res

In [138]:
res_negation_invert = negation_invert(lem_reviews)

In [139]:
accuracy_score(answers, res_negation_invert) # точность

0.6752728799328296